#Image Classification - Task 2

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
%tensorflow_version 2.x
import pandas as pd
import tensorflow as tf
import os
import fnmatch
from os import path
import pickle

print(tf.__version__)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.vision.transform import *
from fastai.metrics import error_rate, accuracy

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

import pandas as pd

2.0.0


In [0]:
bs = 64
np.random.seed(9238)

root_path = F'/content/gdrive/My Drive/data/models/'

# Load Models and Learners

In [0]:
# model 1 ROOM TYPE
with open(root_path + 'task_2_model_2f.pickle', "rb") as input_file:
  data_1 = pickle.load(input_file)
# model 3 a STANDARD
with open(root_path + 'task_3_b_1.pickle', "rb") as input_file:
  data_2 = pickle.load(input_file)
# model 3 b TECHNICAL
#with open(root_path + 'task_3_b_4.pickle', "rb") as input_file:
#  data_3 = pickle.load(input_file)
# model 3 b TECHNICAL V2
with open(root_path + 'task_3_b_1_2.pickle', "rb") as input_file:
  data_3 = pickle.load(input_file)
# model 3 b OBJECTS
with open(root_path + 'task-1.pickle', "rb") as input_file:
  data_4 = pickle.load(input_file)

In [0]:
# model 1 ROOM TYPE learn1
learn1 = cnn_learner(data_1, models.resnet34, metrics=accuracy)
learn1 = learn1.load("/content/gdrive/My Drive/data/models/task_2_model_2f")
# model 3 a STANDARD learn2
learn2 = cnn_learner(data_2, models.resnet50, metrics=accuracy)
learn2 = learn2.load("/content/gdrive/My Drive/data/models/Task_3_b_1")
# model 3 b TECHNICAL learn3
#learn3 = cnn_learner(data_3, models.resnet34, metrics=accuracy)
#learn3 = learn3.load("/content/gdrive/My Drive/data/models/Task_3_b_4")
# model 3 b TECHNICAL V2 learn3
learn3 = cnn_learner(data_3, models.resnet50, metrics=accuracy)
learn3 = learn3.load("/content/gdrive/My Drive/data/models/Task_3_b_1_2")
# model 3 b OBJECTS learn4
learn4 = cnn_learner(data_4, models.resnet50, metrics=accuracy)
learn4 = learn4.load("/content/gdrive/My Drive/data/models/task-1-b")

In [0]:
validation_set = os.listdir('/content/gdrive/My Drive/data/main/')
validation_set = fnmatch.filter(os.listdir('/content/gdrive/My Drive/data/main/'), '*.jpg')
len(validation_set)

29

In [0]:
val_predicts1 = []
val_predicts2 = []
val_predicts3 = []
val_predicts4 = []
image_names = []

for img in validation_set:
  try:
    img_l = open_image("/content/gdrive/My Drive/data/main/"+ img)
    preds1 = learn1.predict(img_l)
    preds2 = learn2.predict(img_l)
    preds3 = learn3.predict(img_l)
    preds4 = learn4.predict(img_l)
    val_predicts1.append(preds1)
    val_predicts2.append(preds2)
    val_predicts3.append(preds3)
    val_predicts4.append(preds4)
    image_names.append(img)
  except:
    print("failed to load image")

In [0]:
task2_class_raw = [item[0] for item in val_predicts1]
task2_class = []
for word in task2_class_raw:
  word = str(word)
  word_cln = word.replace("Category", "")
  task2_class.append(word_cln)

task3a_class_raw = [item[0] for item in val_predicts2]
task3a_class = []
for word in task3a_class_raw:
  word = str(word)
  word_cln = word.replace("Category", "")
  task3a_class.append(word_cln)

task3b_class_raw = [item[0] for item in val_predicts3]
task3b_class = []
for word in task3b_class_raw:
  word = str(word)
  word_cln = word.replace("Category", "")
  task3b_class.append(word_cln)

task4_class_raw = [item[0] for item in val_predicts4]
task4_class = []
for word in task4_class_raw:
  word = str(word)
  word_cln = word.replace("MultiCategory", "")
  word_lst = word_cln.split(";")
  task4_class.append(word_lst)

#len(task2_class)

In [0]:
#Parse response for multiclass object detection
exp = task4_class
uniqe = set([item for sublist in exp for item in sublist])

a = dict()
for pred in uniqe:
  a[pred] = []
#print(a.keys())
for item in exp:
  for item1 in item:
    a[item1].append(1)
  
  not_in_pred = uniqe.difference(item)
  for item2 in not_in_pred:
    a[item2].append(0)

In [0]:
eval = pd.read_csv("/content/gdrive/My Drive/data/eval_sample_output_single.csv")

eval = eval[0:0]

In [0]:
eval['filename'] = image_names
eval['task2_class'] = task2_class
eval['tech_cond'] = task3a_class
eval['standard'] = task3b_class

In [0]:
eval = eval.fillna("0")

In [0]:
results_pred_t1 = pd.DataFrame(a)
# Replace columns
for column in eval.loc[:, "Bathroom":].columns:
  try:
    eval[column] = results_pred_t1[column]
  except:
    pass

In [0]:
eval

,filename,standard,task2_class,tech_cond,Bathroom,Bathroom cabinet,Bathroom sink,Bathtub,Bed,Bed frame,Bed sheet,Bedroom,Cabinetry,Ceiling,Chair,Chandelier,Chest of drawers,Coffee table,Couch,Countertop,Cupboard,Curtain,Dining room,Door,Drawer,Facade,Fireplace,Floor,Furniture,Grass,Hardwood,House,Kitchen,Kitchen & dining room table,Kitchen stove,Living room,Mattress,Nightstand,Plumbing fixture,Property,Real estate,Refrigerator,Roof,Room,Rural area,Shower,Sink,Sky,Table,Tablecloth,Tap,Tile,Toilet,Tree,Urban area,Wall,Window
0,lazienka_4.jpg,3,bathroom,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
1,salon_3.jpg,4,living_room,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,sypialnia_2.jpg,4,bedroom,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,dom_3.jpg,3,house,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,jadalnia_2.jpg,4,dinning_room,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
5,dom_4.jpg,3,living_room,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,kuchnia_2.jpg,3,kitchen,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,dom_5.jpg,2,house,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,jadalnia_5.jpg,3,dinning_room,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,salon_2.jpg,3,living_room,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
eval.to_csv("/content/gdrive/My Drive/data/Final_Results6.csv",index=False)